In [8]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam


In [9]:
image_size = (48, 48)
batch_size = 64

train_ds = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(
    "../data/train",
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)
val_ds = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255).flow_from_directory(
    "../data/test",
    target_size=image_size,
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode="categorical",
)

Found 15743 images belonging to 7 classes.
Found 3756 images belonging to 7 classes.


In [10]:
# def model():
model = tf.keras.Sequential()
lay = tf.keras.layers
model.add(lay.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(48, 48, 1)))
model.add(lay.Conv2D(64, kernel_size=(3, 3), activation="relu"))
model.add(lay.MaxPooling2D(pool_size=(2, 2)))
model.add(lay.Dropout(0.25))

model.add(lay.Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(lay.MaxPooling2D(pool_size=(2, 2)))
model.add(lay.Conv2D(128, kernel_size=(3, 3), activation="relu"))
model.add(lay.MaxPooling2D(pool_size=(2, 2)))
model.add(lay.Dropout(0.25))

model.add(lay.Flatten())
model.add(lay.Dense(1024, activation="relu"))
model.add(lay.Dropout(0.5))
model.add(lay.Dense(7, activation="softmax"))

    # return model

# model()

In [11]:

emotion_dict = {
    0: "Angry",
    1: "Disgusted",
    2: "Fearful",
    3: "Happy",
    4: "Neutral",
    5: "Sad",
    6: "Surprised",
}


In [12]:

model.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(lr=0.0001, decay=1e-6),
    metrics=["accuracy"],
)


/home/abdelmajed/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [13]:
emotion_model_info = model.fit_generator(
    train_ds,
    steps_per_epoch=3709 // 64,
    epochs=1,
    validation_data=val_ds,
    validation_steps=7000 // 64,
)

/home/abdelmajed/.local/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


57/57 [==============================] - 78s 1s/step - loss: 1.3171 - accuracy: 0.4192 - val_loss: 1.1845 - val_accuracy: 0.4723


In [14]:
model.save_weights("emotion_model2.h5")
